idea what the data could look like
(17890902, {'nom': 'Conseils Generaux', 'ville': 'versaille', 'sequenceHistorique': ''})
(timestamp1, {'a':False, 'b':False, 'c':False}),
(timestamp2, {'a':False, 'b':True, 'c':False}),
(timestamp3, {'a':False, 'b':False, 'c':False}), 
(timestamp4, {'a':False, 'b':True, 'c':False}),
https://stackoverflow.com/questions/38465620/looking-for-an-efficient-way-to-store-history-data

### 1) RECUPERATION DU CONTENU DE LA PAGE WIKIPEDIA <br>
https://fr.wikipedia.org/wiki/Chronologie_de_la_R%C3%A9volution_fran%C3%A7aise <br>

In [1]:
# Importer les modules nécessaires
import requests

# Récupérer l'Url
r = requests.get("https://fr.wikipedia.org/wiki/Chronologie_de_la_R%C3%A9volution_fran%C3%A7aise")

# Enregistrer le contenu dans un fichier
with open("response.html", "w") as f:
    f.write(r.text)
    f.close() 


In [97]:
# Importer les modules nécessaires
from bs4 import BeautifulSoup
import json

# Lire le fichier contenant le contenu
with open("response.html", "r") as f:
 
# Parser l'html pour récupérer les <h2>, <h3> et <li> visés
    soup = BeautifulSoup(f, 'html.parser')

periode = []
annee = []
dateEtEvenement = []
dates = soup.find(class_="mw-body-content").find_all("li", class_=False)
for date in dates:
    if date.find_previous("h3") == None:
        periode.append(date.find_previous("h2").text)
        annee.append(0)
        dateEtEvenement.append(date.text)
    else:
        periode.append(date.find_previous("h2").text)
        annee.append(date.find_previous("h3").text)
        dateEtEvenement.append(date.text)

# Sauvegarder les fichiers
with open("periode.json", "w") as f:
    json.dump(periode, f)
    f.close() 
with open("annee.json", "w") as f:
    json.dump(annee, f)
    f.close() 
with open("dateEtEvenement.json", "w") as f:
    json.dump(dateEtEvenement, f)
    f.close() 

### 2) NETTOYAGE DES DONNEES <br>

In [58]:
# Importer les modules nécessaires
import pandas as pd
import json

# Lire les jsons et récupérer les listes
with open("periode.json", "r") as f:
    periode = json.load(f)
with open("annee.json", "r") as f:
    annee = json.load(f)
with open("dateEtEvenement.json", "r") as f:
    dateEtEvenement = json.load(f)
    
# Construire le dataframe 
df = pd.DataFrame ({'Période': periode, 'Année': annee, 'dateEtEvenement': dateEtEvenement})
#print(df.head(10))
#print(df.tail(10))
#print(df.dtypes)
#print(df.describe())
#print(df.info())

# Récupérer les index à supprimer
toDrop = df.loc[(df['Période'] == 'Notes et références') | (df['Période'] == 'Voir aussi')].index
df2 = df.drop(toDrop)
#print(df2.head(10))
#print(df2.tail(10))
#print(df2.dtypes)
#print(df2.describe())
#print(df2.info())

"""
toDrop = df2.loc[(df2['dateEtEvenement'].str.contains('Liste'))].index
df3 = df2.drop(toDrop)
#print(df2.head(10))
#print(df2.tail(10))
#print(df2.dtypes)
#print(df3.describe())
#print(df3.info())

toDrop = df3.loc[(~df3['dateEtEvenement'].str.contains(r'(.*?):'))] #.index
#print(toDrop)
#df3 = df2.drop(toDrop)
#print(df2.head(10))
#print(df2.tail(10))
#print(df2.dtypes)
#print(df3.describe())
#print(df3.info())

dfGood = df3.loc[(df3['dateEtEvenement'].str.contains(r'(.*?):'))] #.index
#print(dfGood)

dfBad = df3.loc[(~df3['dateEtEvenement'].str.contains(r'(.*?):'))] #.index
print(dfBad)


12 septembre 1793 : 
bataille d'Avesnes-le-Sec (en)
combat d'Haspres 


19 septembre : 
319                          bataille du Pont-Barré[7]  
320                              bataille de Torfou[7] 


193                                   combat d'Haspres  
194                     bataille d'Avesnes-le-Sec (en)  
319                          bataille du Pont-Barré[7]  
320                              bataille de Torfou[7]  
454  En février on conseille à Bonaparte de porter ...  
489           6 - 7 novembre bataille de Calliano (en)  
502                        Expédition d'Irlande (1796)  
504                        Expédition d'Irlande (1798)  
"""
"""
#df2[['First','Last']] = df2.dateEtEvenement.str.split(r"(.*?):", n=1) #REGEX IMPORTANT DETERMINER PREMIER :
for i in df['dateEtEvenement']: 
    if i.count(":") == 2:
        print(i)
    if i.count(":") == 0:
        print(i)
    
#find a way to split
#print(df2['Date et Evènement'].str.split(' : ', n=1, expand=True))
#df3.columns = ['column{}'.format(x+1) for x in df3.columns]
#print(df2)
#print(df2['Date et Evènement'].str.split(' : ', 1).tolist(), columns = ['Date', 'Evènement'])

#print(df2)
"""

                                               Période Année  \
193  Convention montagnarde (1793-1794) et Première...  1793   
194  Convention montagnarde (1793-1794) et Première...  1793   
319                    Guerre de Vendée et Chouannerie  1793   
320                    Guerre de Vendée et Chouannerie  1793   
454       Directoire (1795-1798) et Première Coalition  1798   
489                      Campagne d'Italie (1796-1797)  1796   
502              Expéditions d'Irlande de 1796 et 1798  1796   
504              Expéditions d'Irlande de 1796 et 1798  1798   

                                       dateEtEvenement  
193                                   combat d'Haspres  
194                     bataille d'Avesnes-le-Sec (en)  
319                          bataille du Pont-Barré[7]  
320                              bataille de Torfou[7]  
454  En février on conseille à Bonaparte de porter ...  
489           6 - 7 novembre bataille de Calliano (en)  
502                     

/tmp/ipykernel_3735/1326488139.py:38: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  toDrop = df3.loc[(~df3['dateEtEvenement'].str.contains(r'(.*?):'))] #.index
/tmp/ipykernel_3735/1326488139.py:47: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dfGood = df3.loc[(df3['dateEtEvenement'].str.contains(r'(.*?):'))] #.index
/tmp/ipykernel_3735/1326488139.py:50: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dfBad = df3.loc[(~df3['dateEtEvenement'].str.contains(r'(.*?):'))] #.index


'\n#df2[[\'First\',\'Last\']] = df2.dateEtEvenement.str.split(r"(.*?):", n=1) #REGEX IMPORTANT DETERMINER PREMIER :\nfor i in df[\'dateEtEvenement\']: \n    if i.count(":") == 2:\n        print(i)\n    if i.count(":") == 0:\n        print(i)\n    \n#find a way to split\n#print(df2[\'Date et Evènement\'].str.split(\' : \', n=1, expand=True))\n#df3.columns = [\'column{}\'.format(x+1) for x in df3.columns]\n#print(df2)\n#print(df2[\'Date et Evènement\'].str.split(\' : \', 1).tolist(), columns = [\'Date\', \'Evènement\'])\n\n#print(df2)\n'

Source : <br>
https://www.geeksforgeeks.org/web-scraping-from-wikipedia-using-python-a-complete-guide/ <br>

In [ ]:
2) Save data in file <br>
https://stackoverflow.com/questions/31126596/saving-response-from-requests-to-file

In [ ]:
3) Plot as a a timeline <br>
https://matplotlib.org/stable/gallery/lines_bars_and_markers/timeline.html
https://coderzcolumn.com/tutorials/data-science/timeline-using-matplotlib